In [2]:
import pandas

data_token = pandas.read_csv('../Datasets/Train_data_ner_tags.csv')['Tokens'].tolist()
ner_tag = pandas.read_csv('../Datasets/Train_data_ner_tags.csv')['NER_tag'].tolist()


In [3]:
'''
for index in range(0, 8000):
    word_list = eval(data_token[index])
    tag_list = [ner_tag_decoder[i] for i in eval(ner_tag[index])]
    sentence_id_list = [index for i in range(len(word_list))]

    dataframe = pandas.DataFrame({'words': word_list, 'labels': tag_list, 'sentence_id': sentence_id_list})
    dataframe.to_csv('../Datasets/conll_format_train.csv', mode='a', encoding='utf-8', header=False,
                     columns=dataframe.keys(), index=False, index_label=False)
    print(f'current at index {index}')
'''

"\nfor index in range(0, 8000):\n    word_list = eval(data_token[index])\n    tag_list = [ner_tag_decoder[i] for i in eval(ner_tag[index])]\n    sentence_id_list = [index for i in range(len(word_list))]\n\n    dataframe = pandas.DataFrame({'words': word_list, 'labels': tag_list, 'sentence_id': sentence_id_list})\n    dataframe.to_csv('../Datasets/conll_format_train.csv', mode='a', encoding='utf-8', header=False,\n                     columns=dataframe.keys(), index=False, index_label=False)\n    print(f'current at index {index}')\n"

In [4]:
train_df = pandas.read_csv('../Datasets/conll_format_train.csv')
test_df = pandas.read_csv('../Datasets/conll_format_test.csv')

In [5]:
data = [[train_df['sentence_id'].nunique(), test_df['sentence_id'].nunique()]]
pandas.DataFrame(data, columns=['Train', 'Test'])

,Train,Test
0,8000,1580


In [6]:
train_args = {
    'reprocess_input_data': True,
    'overwrite_output_dir': True,
    'sliding_window': True,
    'max_seq_length': 64,
    'num_train_epochs': 5,
    'train_batch_size': 32,
    'fp16': True,
    'output_dir': '/outputs/',
}

In [7]:
custom_label = [i for i in ner_tag_decoder.values()]
custom_label

['B-GPE',
 'I-GPE',
 'B-PER',
 'I-PER',
 'B-DATE',
 'I-DATE',
 'B-ORG',
 'I-ORG',
 'B-CARDINAL',
 'I-CARDINAL',
 'B-NORP',
 'I-NORP',
 'B-LOC',
 'I-LOC',
 'B-TIME',
 'I-TIME',
 'B-FAC',
 'I-FAC',
 'B-MONEY',
 'I-MONEY',
 'B-ORDINAL',
 'I-ORDINAL',
 'B-EVENT',
 'I-EVENT',
 'B-WFA',
 'I-WFA',
 'B-QUANTITY',
 'I-QUANTITY',
 'B-PERCENT',
 'I-PERCENT',
 'B-LANGUAGE',
 'I-LANGUAGE',
 'B-PRODUCT',
 'I-PRODUCT',
 'B-LAW',
 'I-LAW',
 'O']

In [8]:
from simpletransformers.ner import NERModel
from transformers import AutoTokenizer
import logging

logging.basicConfig(level=logging.DEBUG)
transformers_logger = logging.getLogger('transformers')
transformers_logger.setLevel(logging.WARNING)

# We use the bert base cased pre-trained model.
tokenizer = AutoTokenizer.from_pretrained('bert-base-chinese')
model = NERModel('bert', 'bert-base-chinese', args=train_args, labels=custom_label, use_cuda=True, cuda_device=0)


DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /bert-base-chinese/resolve/main/tokenizer_config.json HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /bert-base-chinese/resolve/main/config.json HTTP/1.1" 200 0
Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForTokenClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializin

In [9]:
model.train_model(train_df, output_dir='../model_output')

INFO:simpletransformers.ner.ner_model: Converting to features started.


  0%|          | 0/16 [00:00<?, ?it/s]

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Running Epoch 0 of 5:   0%|          | 0/250 [00:00<?, ?it/s]

C:\Users\asdf5\Desktop\Named-Entity-Recognition\venv\Lib\site-packages\torch\optim\lr_scheduler.py:139: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


...

Running Epoch 2 of 5:   0%|          | 0/250 [00:00<?, ?it/s]

Running Epoch 3 of 5:   0%|          | 0/250 [00:00<?, ?it/s]

Running Epoch 4 of 5:   0%|          | 0/250 [00:00<?, ?it/s]

INFO:simpletransformers.ner.ner_model: Training of bert model complete. Saved to ../model_output.


(1250, 0.15967384540736676)

In [10]:
result, model_outputs, preds_list = model.eval_model(test_df)

INFO:simpletransformers.ner.ner_model: Converting to features started.


  0%|          | 0/4 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/198 [00:00<?, ?it/s]

INFO:simpletransformers.ner.ner_model:{'eval_loss': 0.13390802778303623, 'precision': 0.6987855544902525, 'recall': 0.8016498625114574, 'f1_score': 0.746691710065739}


In [11]:
samples = ['我 住 在 雲 林 ， 這 很 可 以']
predictions, _ = model.predict(samples)
for idx, sample in enumerate(samples):
    print('{}: '.format(idx))
    for word in predictions[idx]:
        print('{}'.format(word))

INFO:simpletransformers.ner.ner_model: Converting to features started.


  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

0: 
{'我': 'O'}
{'住': 'O'}
{'在': 'O'}
{'雲': 'B-PER'}
{'林': 'I-PER'}
{'，': 'O'}
{'這': 'O'}
{'很': 'O'}
{'可': 'O'}
{'以': 'O'}


True